по терминологии: чтобы не возникло путаницы в понятии вершины, я буду называть вершину графа вертексом, а вершину в синтаксическом дереве вершиной
а ребро графа либо ребром, либо эджом. чанк - это именная группа

In [2]:
import spacy
import spacy.cli
from graph import Graph, visualize_graph
from edge import Edge
from vertex import Vertex
from spacy import displacy

In [68]:
spacy.cli.download("en_core_web_lg")

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 6.9 MB/s eta 0:00:0000:0100:03
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
def process_token(token, graph, chunk_to_vertex):
    """
    Обрабатывает токен, добавляя рёбра между существующими вершинами (именными группами).
    """
    for child in token.children:
        head_vertex = chunk_to_vertex.get(token)
        child_vertex = chunk_to_vertex.get(child)
        
        # Добавляем рёбра только если обе вершины существуют и токен является глаголом, предлогом или союзом
        if head_vertex and child_vertex and token.pos_ in {"VERB", "ADP", "CCONJ"}:
            graph.add_edge(head_vertex.concept, child_vertex.concept, token.text, 1, 0)
        
        process_token(child, graph, chunk_to_vertex)  # Рекурсия для потомков

In [77]:
def build_graph_from_parse_tree(doc) -> Graph:
    """
    Строит граф, где вершины - только именные группы (noun chunks),
    а рёбра - глаголы, предлоги или союзы.
    """
    graph = Graph()
    chunk_to_vertex = {}
    
    # Добавляем вершины только для именных групп, исключая глаголы
    for chunk in doc.noun_chunks:
        if chunk.root.pos_ not in {"VERB"}:  # Исключаем глаголы
            vertex = Vertex(chunk.text, [chunk.text])
            graph.add_vertex(vertex.concept, vertex.words_of_concept)
            chunk_to_vertex[chunk.root] = vertex  # Привязываем root токен к вершине
    
    # Добавляем рёбра между существующими вершинами
    for token in doc:
        if token.pos_ in {"VERB", "ADP", "CCONJ"}:  # Обрабатываем только рёбра
            process_token(token, graph, chunk_to_vertex)
    
    return graph

In [71]:
text = "The cat on the mat sees the dog and barks loudly."
doc = nlp(text)
syntax_graph = build_graph_from_parse_tree(doc)
print("Vertices:", syntax_graph.vertices)
print("Edges:", syntax_graph.edges)
visualize_graph(syntax_graph)

NameError: name 'nlp' is not defined

In [65]:
displacy.render(doc, style='dep')